In [46]:
import pandas as pd
import plotly.express as px 
import scipy
import numpy as np
import plotly.io as pio

In [127]:
# set various parameters
car_price = 65000
# start_cap = 140000
maintenance_monthly = 500*12*10

In [128]:
# create range of downpayments possible
downpayment_list = [i for i in range(int(car_price*0.3),car_price,200)]

In [129]:
# create dataframe
car_loan_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# add total cost of car to dataframe
for z in range(1,8):
    car_loan_df["{} Year".format(z)] = [((0.0298*z*(car_price - downpayment_list[i])+(car_price - downpayment_list[i])))+downpayment_list[i]+maintenance_monthly for i in range(0,len(downpayment_list))]

# melt for easier plotting
car_melt = pd.melt(car_loan_df, id_vars="Downpayment")

In [50]:
fig = px.line(car_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Total Cost of Car",
                     "variable": "Loan Term"
                 },title="True Cost of Car with Interest and Monthly Payment",
                 color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.show()

In [80]:
# function to calculate compound interest given monthly contributions
def fv(monthly, interest_rate, time):
    return (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate)))

# function to calculate compound interest given monthly contribution and starting amount
def ifv(initial, monthly, interest_rate, time):
    return ((initial*((1+interest_rate)**time)) + (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate))))

In [52]:
# function to calculate the return on investment given 10 year capital to ivnest
def car_tradeoff_calc(car_price,downpayment,years,bank_int,capital_invest,simir):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    pre_invest_monthly = max(0,((capital_invest-downpayment)/(10*12)-pay_monthly))
    pre_invest_total = fv(pre_invest_monthly, simir, years*12)
    post_invest_monthly = ((capital_invest-((pay_monthly+ pre_invest_monthly)*years*12)-downpayment)/((10-years)*12))
    post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
    total_returns = pre_invest_total + post_invest_total
    return(post_invest_total)

In [53]:
# create dataframe
payoff_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# make sure one has enough capital to pay for the car fully
if (start_cap < car_loan_df.iloc[0,-1]):
    print("Error: Starting capital not large enough")
else:
    for z in range(1,8):
        payoff_df["{} yr".format(z)] = [car_tradeoff_calc(car_price,downpayment_list[i],z,0.0298,start_cap,0.007) for i in range(0,len(downpayment_list))]

#melt it for easier plotting
payoff_df_melt = pd.melt(payoff_df, id_vars="Downpayment")

In [54]:
fig = px.line(payoff_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with 10 year capital: {}".format(start_cap))
fig.show()

In [126]:
# function to calculate the return on investment given monthly budget
def car_tradeoff_calc_monthly(car_price,downpayment,years,bank_int,cap_month,simir, maintenance_monthly):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    if cap_month < pay_monthly:
        loss = (pay_monthly - cap_month)*years*12
        post_invest_monthly = cap_month
        post_invest_total = fv(post_invest_monthly, simir, ((10-years)*12))
        total_returns = post_invest_total - loss
        return (total_returns)
    else: 
        pre_invest_monthly = max(0,(cap_month-pay_monthly))
        pre_invest_total = fv(pre_invest_monthly, simir, years*12)
        post_invest_monthly = cap_month
        post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
        total_returns = pre_invest_total + post_invest_total
        return(post_invest_total)

month_cap = 800

In [131]:
#create dataframe
payoffmonth_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# add return on investments into dataframe
for z in range(1,8):
    payoffmonth_df["{} yr".format(z)] = [car_tradeoff_calc_monthly(car_price,downpayment_list[i],z,0.0298,month_cap,0.007,maintenance_monthly) for i in range(0,len(downpayment_list))]

# melt it for easier plotting
payoffmonth_df_melt = pd.melt(payoffmonth_df, id_vars="Downpayment")

In [163]:
fig = px.line(payoffmonth_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with monthly budget: {}".format(month_cap))

fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.add_shape(type="line",
    x0=20000, y0=0, x1=downpayment_list[-1], y1=0,
    line=dict(color="Black",width=1)
)
fig.show()